# Homework: turn the FakeNews text column into tfidf vectors the easy way
  
Vocab:
- corpus: set of all documents in a dataset

## `news.csv` Data Set

4 columns: 
- article id
- article title
- article text
- lable

In [13]:
import itertools
import re

from collections import Counter

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

In [14]:
#Read the data
df=pd.read_csv('data/news.csv')

#Get shape and head
shape = df.shape
print(f"shape of the dataset: {shape} \n")

df.head(10)

shape of the dataset: (6335, 4) 



,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
5,6903,"Tehran, USA","\nI’m not an immigrant, but my grandparents ...",FAKE
6,7341,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",FAKE
7,95,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,REAL
8,4869,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,REAL
9,2909,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,REAL


## Machine Learning Framework API Patterns: Transformers and Estimators

Machine learning pipelines take a raw dataset and then transform the data over a series of steps and finally outputs some prediction. For example, text documents -> tokenization -> word counts -> idf calculation -> tfidf calculation -> build classification model -> predictions

The individual steps in a pipeline are called Transformers. Some Transformers can take the input data and directly transform it without any understanding of the full dataset. For example, a stopword filter transformer might take a set of document token arrays, and remove stopwords from each document token array.

Transformers usually have a `transform()` function to do it's work. 

Other types of Transformers need to capture some understanding of the entire dataset before it can Transform the input data. For example a min/max normalization transformer needs to pass over the entire data set once to figure out the minimum and maximum values, then it'll pass over the dataset a second time and calculate the min/max normalization. 

These types of Transformers are called Estimators and usually have a `fit()` function which will do the first pass over the data and calculate it's required state (min and max values), and returns a Transformer that has this state. Then you can call `transform()` on this object to transform the data (calc min/max norms)

Scikit Learn, Spark, and a few others I've run across follow this pattern.

## Word Tokenization & Token Counts

CountVectorizer: Converts a collection of text documents into a matrix of token counts

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [15]:
# example
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['this is the first document',
           'this document is the second document',
           'and this is the third one',
           'is this the first document',         
         ]

vectorizer = CountVectorizer()

estimator = vectorizer.fit(corpus)
X_train_counts = estimator.transform(corpus)

X_train_counts = vectorizer.fit_transform(corpus)

# dictionary of all terms in corpus
#     Warning: term overload...this does not mean a python dict, it's a dictionary in the "list of words" sense.
print(vectorizer.get_feature_names())

# token count marix from corpus
X_train_counts.toarray()

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


array([[0, 1, 1, 1, 0, 0, 1, 0, 1],
       [0, 2, 0, 1, 0, 1, 1, 0, 1],
       [1, 0, 0, 1, 1, 0, 1, 1, 1],
       [0, 1, 1, 1, 0, 0, 1, 0, 1]])

### Task: Run the text column from the FakeNews dataset through the CountVectorizer

Task 1a:
- turn the raw "text" column into a count vector
  - Try to keep all data in the pandas dataframe. So when creating the count vectors, put them back into the pandas dataframe as a new column
- what is the size of the corpus dictionary?
- what is the shape of the fitted document token count matrix
- from a TF-IDF perspective, how does the output of the CountVectorizer relate to the TF-IDF calculation

Task 1b:
- look at the CountVectorizer documentation and find the `min_df` and `max_df` parameters to the CountVectorizer constructor
  - use these params to filter out terms (tokens) that are only in 5 documents or less
  - use these params to filter out terms (tokens) that are in 95% of the docuemnts or more
- what is the size of the corpus dictionary?
- why are these two types of term filters useful?

In [16]:
# build count matrix for Fake News dataset, put it back on the data frame as a new field
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(df["text"])


In [17]:
df['count_vector'] = counts

In [18]:
df.head(5)

,Unnamed: 0,title,text,label,count_vector
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,"(0, 15786)\t1\n (0, 26358)\t1\n (0, 54359)..."
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,"(0, 15786)\t1\n (0, 26358)\t1\n (0, 54359)..."
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,"(0, 15786)\t1\n (0, 26358)\t1\n (0, 54359)..."
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,"(0, 15786)\t1\n (0, 26358)\t1\n (0, 54359)..."
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,"(0, 15786)\t1\n (0, 26358)\t1\n (0, 54359)..."


out of curiousity, what's at the top and bottom of the dictionary?

In [19]:
vectorizer.get_feature_names()[:30]

['00',
 '000',
 '0000',
 '000000031',
 '00000031',
 '000035',
 '00006',
 '0001',
 '0001pt',
 '0002',
 '000billion',
 '000ft',
 '000km',
 '000x',
 '001',
 '0011',
 '002',
 '003',
 '004',
 '004s',
 '005',
 '005s',
 '006',
 '00684',
 '006s',
 '007',
 '007s',
 '008',
 '008s',
 '009']

numbers. That's not super surprising, I guess.

In [20]:
vectorizer.get_feature_names()[-30:]

['שני',
 'שעת',
 'שתי',
 'תאמצנה',
 'תוצאה',
 'תחל',
 'תיירות',
 'תנותק',
 'תעודת',
 'תתרכז',
 'أن',
 'إجلاء',
 'الأمر',
 'الجرحى',
 'الدولية',
 'القادمون',
 'اللجنة',
 'تحتاج',
 'تعرفه',
 'تنجح',
 'حلب',
 'عربي',
 'عن',
 'لم',
 'ما',
 'محاولات',
 'من',
 'هذا',
 'والمرضى',
 'ยงade']

WTF? 

I had assumed that the articles in the dataset were pure English. That's clearly not true. Reading the entries, it looks like the posts containing these words are quoting people who posted in those languages. I'm not sure this changes anything about the analysis so far, but will have to consider that if we do English-specific things to the text later, it might do surprising things to Arabic or Hebrew.

In [21]:
# what's the size of the corpus dictionary?
print(f"feature dict length: {len(vectorizer.get_feature_names())}")
# what's the shape of the counts matrix
print(f"{counts.shape}")

feature dict length: 67659
(6335, 67659)


#### from a TF-IDF perspective, how does the output of the CountVectorizer relate to the TF-IDF calculation

In the manual computation of the TF-IDF, I had a "term_count", which was the number of times a term occurred in the given doc, and the tf was then just the term_count of that term divided by the # of terms in the doc. The CountVectorizer is that term_count, precomputed. To get the TF, you'll need to sum the whole vector to get the total # of terms in the document, but that's fairly trivial for a vector of numbers. 

To get the IDF for a given term, you could sum across the docs at a fixed point in the vector, corresponding to the term you're looking for. So, the CountVectorizer is pre-computing a few of the steps needed for TF-IDF.


## Create TFIDF Vectors

TfidfTransformer: Converts a count matrix into a tfidf matrix

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html

In [22]:
from sklearn.feature_extraction.text import TfidfTransformer

tf_transformer = TfidfTransformer(use_idf=True, smooth_idf=True).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)

# dictionary term idf values
print(tf_transformer.idf_)

# token count marix from corpus
X_train_tf.toarray()

[1.91629073 1.22314355 1.51082562 1.         1.91629073 1.91629073
 1.         1.91629073 1.        ]


array([[0.        , 0.46979139, 0.58028582, 0.38408524, 0.        ,
        0.        , 0.38408524, 0.        , 0.38408524],
       [0.        , 0.6876236 , 0.        , 0.28108867, 0.        ,
        0.53864762, 0.28108867, 0.        , 0.28108867],
       [0.51184851, 0.        , 0.        , 0.26710379, 0.51184851,
        0.        , 0.26710379, 0.51184851, 0.26710379],
       [0.        , 0.46979139, 0.58028582, 0.38408524, 0.        ,
        0.        , 0.38408524, 0.        , 0.38408524]])

### Task: Run the token count data (that used the `min_df` and `max_df` parameters) from the FakeNews dataset through the TfidfTransformer

- What's different between the counts matrix and the tfidf matrix?

so much easier than manually calculating tfidf right?

In [31]:
# Stuck here. Can do by hand like:

#fake_news_vectorizer = CountVectorizer()

#fake_train_counts = fake_news_vectorizer.fit_transform(df['text'])

# but that's cheating, isn't it? I should be using the counts off of the dataframe itself, yes?

# if I try 
# df['count_vector'] = counts
#  
# I get an exception that talks about a float conversion
#
# if I try
# df['count_vector'] = [count for count in counts]
# I get a also get the float conversion exception 
#
# If I try
df['count_vector'] = [ar for ar in counts.toarray()]
# I get a third exception that talks about size-1 arrays.
TfidfTransformer(use_idf=True).fit_transform(df['count_vector'])

#
# In theory I could just keep going with fake_train_counts, but you specifically asked me to keep as much
# as possible in the dataframe, and everything I try to put it there is causing exceptions, so I'm a bit stuck.
#

ValueError: setting an array element with a sequence.

In [59]:
# calculate tfidf for the text column in the FakeNews dataset





## Machine Learning Pipelines

Most ML frameworks have a pipeline framework, where you can add multiple different transformers into a parent transformer, then you only all `fit()` and `transform()` on the pipeline object. Internally the pipeline will call `fit()` and `transform()` on each individual transformer and output the final matrix of data

In [10]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('count', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
]).fit(corpus)

X = pipe.transform(corpus)
X.toarray()

array([[0.        , 0.46979139, 0.58028582, 0.38408524, 0.        ,
        0.        , 0.38408524, 0.        , 0.38408524],
       [0.        , 0.6876236 , 0.        , 0.28108867, 0.        ,
        0.53864762, 0.28108867, 0.        , 0.28108867],
       [0.51184851, 0.        , 0.        , 0.26710379, 0.51184851,
        0.        , 0.26710379, 0.51184851, 0.26710379],
       [0.        , 0.46979139, 0.58028582, 0.38408524, 0.        ,
        0.        , 0.38408524, 0.        , 0.38408524]])

In [11]:
print(f"count vectorizer dictionary: {pipe['count'].get_feature_names()}")
print()
print(f"tfidf transformer's idf data: {pipe['tfidf'].idf_}")

count vectorizer dictionary: ['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']

tfidf transformer's idf data: [1.91629073 1.22314355 1.51082562 1.         1.91629073 1.91629073
 1.         1.91629073 1.        ]


### Task: Build a Pipeline to generate a tfidf document matrix



In [64]:
# build and a ML pipeline to calc tfidf on the text column of the FakeNews dataset




## Extra Credit: Text Normalization

There are different algorithms for text normalization, such as stemming and lemitization. These algorithms aren't build into scikit-learn, but other text processing libraries like `nltk` have implementations. Both have pros and cons. I really like lemitization, but it has a heavier processing cost. 

Pick one, stemming or lemitization, and integrate it into your raw text to tfidf pipeline (there are articles out there on how to integrate `nltk` into a scikit-learn pipeline).  

How did this change the vocabulary size?

In [6]:
5^5

0